In this example you will implement and evaluate bagging for a classification
problem.
1. Load the Breast Cancer dataset from the mlbench package.
2. Split the data into training and testing sets.
3. Implement bagging using the ipred package to create an ensemble of classification trees.
4. Evaluate the performance of the bagged model using accuracy, precision, recall, and F1-
score.
5. Compare the performance of the bagged model with that of a single decision tree.

In [1]:
# Bagging vs Single Tree (Python equivalent of mlbench/ipred)
# Dataset: scikit-learn's Wisconsin Diagnostic Breast Cancer (binary classification)
# Metrics: accuracy, precision, recall, F1 (positive class = malignant)

import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Load data (like mlbench::BreastCancer; here we use sklearn's WDBC)
data = load_breast_cancer()
X = data.data
# Map to 1=malignant, 0=benign so positive class is malignant
y = (data.target == 0).astype(int)  # in sklearn: 0=malignant, 1=benign

# Train/test split
X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)

# Models
# Single CART-like tree (unpruned/high-variance is typical baseline for bagging)
tree = DecisionTreeClassifier(
    criterion="gini",
    max_depth=None,
    min_samples_leaf=1,
    random_state=42
)
tree.fit(X_tr, y_tr)

# Bagging of trees (Python analog of ipred::bagging)
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(
        criterion="gini", max_depth=None, min_samples_leaf=1
    ),
    n_estimators=200,
    bootstrap=True,
    oob_score=True,          # out-of-bag estimate like ipred
    n_jobs=-1,
    random_state=42
)
bag.fit(X_tr, y_tr)

# Evaluate helpers
def metrics(y_true, y_prob_or_pred):
    # accept probability or hard labels
    if y_prob_or_pred.ndim == 1 and set(np.unique(y_prob_or_pred)) <= {0,1}:
        y_pred = y_prob_or_pred
    else:
        y_pred = (y_prob_or_pred >= 0.5).astype(int)
    return dict(
        accuracy = accuracy_score(y_true, y_pred),
        precision= precision_score(y_true, y_pred, zero_division=0),
        recall   = recall_score(y_true, y_pred, zero_division=0),
        f1       = f1_score(y_true, y_pred, zero_division=0),
    )

# Predictions
tree_pred = tree.predict(X_te)
bag_pred  = bag.predict(X_te)

tree_metrics = metrics(y_te, tree_pred)
bag_metrics  = metrics(y_te, bag_pred)

# Print comparison
print("=== Single Decision Tree (test) ===")
for k,v in tree_metrics.items():
    print(f"{k:9s}: {v:.4f}")
print("\nClassification report (Tree):")
print(classification_report(y_te, tree_pred, target_names=["benign(0)","malignant(1)"], zero_division=0))

print("\n=== Bagged Trees (test) ===")
for k,v in bag_metrics.items():
    print(f"{k:9s}: {v:.4f}")
print(f"OOB accuracy (Bagging): {bag.oob_score_:.4f}")
print("\nClassification report (Bagging):")
print(classification_report(y_te, bag_pred, target_names=["benign(0)","malignant(1)"], zero_division=0))

# Optional: compact side-by-side summary
print("\n=== Summary ===")
print(f"{'Model':18s}  Acc    Prec   Recall  F1")
print(f"{'Decision Tree':18s}  {tree_metrics['accuracy']:.3f}  {tree_metrics['precision']:.3f}  {tree_metrics['recall']:.3f}  {tree_metrics['f1']:.3f}")
print(f"{'Bagging (200 trees)':18s}  {bag_metrics['accuracy']:.3f}  {bag_metrics['precision']:.3f}  {bag_metrics['recall']:.3f}  {bag_metrics['f1']:.3f}")

=== Single Decision Tree (test) ===
accuracy : 0.9580
precision: 0.9796
recall   : 0.9057
f1       : 0.9412

Classification report (Tree):
              precision    recall  f1-score   support

   benign(0)       0.95      0.99      0.97        90
malignant(1)       0.98      0.91      0.94        53

    accuracy                           0.96       143
   macro avg       0.96      0.95      0.95       143
weighted avg       0.96      0.96      0.96       143


=== Bagged Trees (test) ===
accuracy : 0.9650
precision: 1.0000
recall   : 0.9057
f1       : 0.9505
OOB accuracy (Bagging): 0.9554

Classification report (Bagging):
              precision    recall  f1-score   support

   benign(0)       0.95      1.00      0.97        90
malignant(1)       1.00      0.91      0.95        53

    accuracy                           0.97       143
   macro avg       0.97      0.95      0.96       143
weighted avg       0.97      0.97      0.96       143


=== Summary ===
Model               Acc 

Observation/comparision
1. Accuracy: 0.958 to 0.965 (increased)
2. Precision (malignant as positive): 0.980 to 1.000 (increased — no false positives)
3. Recall: 0.906 to 0.906 (almost same)
4. F1: 0.941 to 0.951 (increased)
5. Implied confusion matrices (test n=143; 90 benign, 53 malignant)
6. Tree: TP≈48, FN≈5, FP≈1, TN≈89 and accuracy (48+89)/143≈0.958.
7. Bagging: TP≈48, FN≈5, FP=0, TN=90 and accuracy (48+90)/143≈0.965.
•	Interpretation
	•	Bagging averages many unstable trees hence variance reduction.
		Here it eliminated the single false positive (hence precision=1.0) while the same hard malignant cases remained misclassified by most trees (recall unchanged).
8. OOB accuracy 0.955 is close to test 0.965 hence out-of-bag gives a trustworthy generalization estimate.
•	What to do next (if recall matters more)
	•	Use probability averaging from bagging and lower the decision threshold (<0.5) to trade a little precision for higher recall.
	•	Add class weights (class_weight='balanced') or stratified bootstraps to target malignant sensitivity.
	•	Consider Random Forests (bagging + feature subsampling) to further decorrelate trees, or boosting to reduce bias on hard cases.

Takeaway: Bagging improved overall performance by removing spurious malignant calls (higher precision and F1) with essentially the same sensitivity; classic variance-reduction behavior.

Construct a regression simulation that will be used for studying the effect of
nodesize on RF performance.
1. Simulate the data and run a RF regression analysis under different values of nodesize.
Determine the optimal nodesize value by minimizing the estimated generalization error of
the OOB ensemble.
2. Using simulated data sets repeat part 1 but use test set estimated generalization error to
obtain the optimal nodesize.
3. Compare your results of parts 2 and 3 and discuss your findings.

In [3]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Simulation (Friedman-1)

def simulate_friedman1(n=3000, p=10, sigma=1.0, rng=None):
    """
    Returns X (n x p), y with Friedman-1 signal on first 5 dims + Gaussian noise.
    """
    rng = np.random.default_rng(rng)
    X = rng.uniform(0.0, 1.0, size=(n, p))
    y = (10*np.sin(np.pi*X[:,0]*X[:,1]) +
         20*(X[:,2]-0.5)**2 +
         10*X[:,3] +
         5*X[:,4] +
         rng.normal(0.0, sigma, size=n))
    return X, y

# One-split evaluation
def sweep_nodesize_once(
    n=3000, p=10, sigma=1.0, test_size=0.3, seed=0,
    nodesize_grid=(1, 2, 5, 10, 20, 30, 50),
    rf_kwargs=None
):
    """
    Simulate, split, fit a RandomForest for each nodesize.
    Returns a DataFrame with OOB and test metrics and the chosen nodesize by each criterion.
    """
    rf_kwargs = rf_kwargs or {}
    X, y = simulate_friedman1(n=n, p=p, sigma=sigma, rng=seed)
    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=test_size, random_state=seed)

    rows = []
    for nodesize in nodesize_grid:
        rf = RandomForestRegressor(
            n_estimators=500,
            bootstrap=True,
            oob_score=True,            # enable OOB predictions
            min_samples_leaf=nodesize, # <-- "nodesize"
            max_features="sqrt",
            random_state=seed,
            n_jobs=-1,
            **rf_kwargs
        )
        rf.fit(X_tr, y_tr)

        # OOB MSE from OOB predictions
        oob_pred = rf.oob_prediction_
        # oob_prediction_ has length len(X_tr); compute its MSE vs y_tr
        oob_mse = float(np.mean((y_tr - oob_pred)**2))

        # Test MSE
        te_pred = rf.predict(X_te)
        te_mse = float(mean_squared_error(y_te, te_pred))

        rows.append({
            "nodesize": nodesize,
            "oob_mse": oob_mse,
            "test_mse": te_mse
        })

    df = pd.DataFrame(rows).sort_values("nodesize").reset_index(drop=True)
    # winners by criterion
    idx_oob  = int(df["oob_mse"].idxmin())
    idx_test = int(df["test_mse"].idxmin())
    out = {
        "table": df,
        "oob_best_nodesize": int(df.loc[idx_oob, "nodesize"]),
        "oob_best_oob_mse": float(df.loc[idx_oob, "oob_mse"]),
        "oob_best_test_mse": float(df.loc[idx_oob, "test_mse"]),   # generalization if selected by OOB
        "test_best_nodesize": int(df.loc[idx_test, "nodesize"]),
        "test_best_test_mse": float(df.loc[idx_test, "test_mse"]),
        "seed": seed
    }
    return out

# 3) Replicated experiment

def run_experiment(
    reps=20, n=3000, p=10, sigma=1.0, nodesize_grid=(1,2,5,10,20,30,50), test_size=0.3
):
    summary_rows = []
    per_seed_tables = []

    for s in range(reps):
        res = sweep_nodesize_once(
            n=n, p=p, sigma=sigma, seed=101 + s,
            nodesize_grid=nodesize_grid, test_size=test_size
        )
        per_seed_tables.append(res["table"].assign(seed=res["seed"]))
        summary_rows.append({
            "seed": res["seed"],
            "oob_best_nodesize": res["oob_best_nodesize"],
            "oob_best_test_mse": res["oob_best_test_mse"],
            "test_best_nodesize": res["test_best_nodesize"],
            "test_best_test_mse": res["test_best_test_mse"],
        })

    perf = pd.DataFrame(summary_rows)
    counts_oob  = perf["oob_best_nodesize"].value_counts().sort_index()
    counts_test = perf["test_best_nodesize"].value_counts().sort_index()

    print("\n=== Selection frequencies over seeds ===")
    print("OOB-selected nodesize:\n", counts_oob)
    print("\nTest-selected nodesize:\n", counts_test)

    print("\n=== Average test MSE (lower is better) ===")
    print("When selecting by OOB: ", perf["oob_best_test_mse"].mean())
    print("When selecting by Test:", perf["test_best_test_mse"].mean())

    # Return for further analysis/plotting if desired
    return perf, pd.concat(per_seed_tables, ignore_index=True)

# Example run
if __name__ == "__main__":
    # Grid is wide enough to show variance-bias tradeoff
    GRID = (1, 2, 5, 10, 20, 30, 50, 80, 120)

    # Single run (Part 1 + Part 2 on one split)
    single = sweep_nodesize_once(nodesize_grid=GRID, seed=123, sigma=1.0)
    print("\n=== One split (Part 1 vs Part 2) ===")
    print(single["table"])
    print("\nOOB-picked nodesize:", single["oob_best_nodesize"],
          "| Test MSE at that pick:", single["oob_best_test_mse"])
    print("Test-picked nodesize:", single["test_best_nodesize"],
          "| Test MSE at that pick:", single["test_best_test_mse"])

    # Replicates (Part 3)
    perf, tables = run_experiment(reps=20, nodesize_grid=GRID, sigma=1.0)
    # Optional: save tables to CSV for plotting externally
    # tables.to_csv("rf_nodesize_sweep.csv", index=False)


=== One split (Part 1 vs Part 2) ===
   nodesize   oob_mse  test_mse
0         1  3.591798  3.334139
1         2  3.663401  3.400498
2         5  4.017821  3.731814
3        10  4.545869  4.259320
4        20  5.358853  5.062020
5        30  6.039767  5.739885
6        50  7.038834  6.720530
7        80  8.320833  7.994824
8       120  9.715746  9.377452

OOB-picked nodesize: 1 | Test MSE at that pick: 3.3341385116016804
Test-picked nodesize: 1 | Test MSE at that pick: 3.3341385116016804

=== Selection frequencies over seeds ===
OOB-selected nodesize:
 oob_best_nodesize
1    20
Name: count, dtype: int64

Test-selected nodesize:
 test_best_nodesize
1    20
Name: count, dtype: int64

=== Average test MSE (lower is better) ===
When selecting by OOB:  3.397484452667908
When selecting by Test: 3.397484452667908


Observation
1. Both OOB and test pick nodesize = 1 (deepest trees).
2. MSE increases monotonically as nodesize grows so larger leaves = more bias.
3. OOB MSE is consistently a bit higher than test MSE (expected: OOB uses ~63% unique samples per tree and less data per prediction).
4. Your sim (Friedman-1, n=3000, sigma=1, 500 trees, bootstrap, max_features=√p) is variance-controlled which is aggregation of bagging, many trees, plenty of data tame the variance of deep trees.
5. With variance already small, pushing nodesize up only raises bias, so test error worsens.
6. When the optimum shifts upward
7. Higher noise (sigma high), smaller n, fewer trees, or more aggressive feature subsampling can make deep trees too wiggly, then the best nodesize typically moves to 5–30.
8. We'll also see a clearer U-shape in MSE vs nodesize under those settings.
9. In our current high-signal, large-n setting, RF benefits from maximally deep trees; OOB selection matches test selection, so OOB is a good proxy for nodesize tuning here.